In [1]:
import pandas as pd
import wget
import os

# Download the file (only if it doesn't already exist)
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
if not os.path.exists("spacex_launch_dash.csv"):
    wget.download(url, "spacex_launch_dash.csv")

# Load it
spacex_df = pd.read_csv("spacex_launch_dash.csv")
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [2]:
launch_sites = spacex_df['Launch Site'].unique().tolist()

In [3]:
options = [{'label': 'All Sites', 'value': 'ALL'}]
for site in launch_sites:
    options.append({'label': site, 'value': site})

In [4]:
from dash import Dash, dcc, html
import pandas as pd
import plotly.express as px

# Load data
spacex_df = pd.read_csv("spacex_launch_dash.csv")

# Extract unique sites
launch_sites = spacex_df['Launch Site'].unique().tolist()

# Build dropdown options
options = [{'label': 'All Sites', 'value': 'ALL'}]
for site in launch_sites:
    options.append({'label': site, 'value': site})

# Create Dash app
app = Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard",
            style={'textAlign': 'center', 'color': '#503D36',
                   'font-size': 40}),

    dcc.Dropdown(
        id='site-dropdown',
        options=options,
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    )
])

# Run inline
app.run(mode="inline")

In [5]:
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard",
            style={'textAlign': 'center', 'color': '#503D36',
                   'font-size': 40}),

    dcc.Dropdown(
        id='site-dropdown',
        options=options,
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),

    html.Br(),  # just for spacing

    # TASK 2: Pie chart placeholder
    dcc.Graph(id='success-pie-chart')
])

In [6]:
from dash.dependencies import Input, Output
import plotly.express as px

@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        # Use whole dataset: show total success counts by site
        fig = px.pie(spacex_df, 
                     values='class', 
                     names='Launch Site', 
                     title='Total Success Launches by Site')
    else:
        # Filter data for the selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(filtered_df, 
                     names='class', 
                     title=f'Success vs Failure for site {entered_site}')
    return fig

In [7]:
app.run(mode="inline")

In [8]:
min_payload = spacex_df['Payload Mass (kg)'].min()
max_payload = spacex_df['Payload Mass (kg)'].max()
print(min_payload, max_payload)  # just to check

0.0 9600.0


In [9]:
html.Br(),

html.P("Payload range (Kg):"),

dcc.RangeSlider(
    id='payload-slider',
    min=0, 
    max=10000, 
    step=1000,
    marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
    value=[min_payload, max_payload]
),

(RangeSlider(min=0, max=10000, step=1000, marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'}, value=[0.0, 9600.0], id='payload-slider'),)

In [10]:
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard",
            style={'textAlign': 'center', 'color': '#503D36',
                   'font-size': 40}),

    # Dropdown (TASK 1)
    dcc.Dropdown(
        id='site-dropdown',
        options=options,
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),

    html.Br(),

    # Pie chart (TASK 2)
    dcc.Graph(id='success-pie-chart'),

    html.Br(),

    # RangeSlider (TASK 3)
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(
        id='payload-slider',
        min=0, 
        max=10000, 
        step=1000,
        marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
        value=[min_payload, max_payload]
    )
])

In [11]:
app.run(mode="inline")

In [12]:
html.Br(),

dcc.Graph(id='success-payload-scatter-chart')

Graph(id='success-payload-scatter-chart')

In [13]:
from dash.dependencies import Input, Output
import plotly.express as px

@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [
        Input(component_id='site-dropdown', component_property='value'),
        Input(component_id='payload-slider', component_property='value')
    ]
)
def get_scatter_chart(selected_site, payload_range):
    low, high = payload_range
    # filter dataset by payload range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) &
                            (spacex_df['Payload Mass (kg)'] <= high)]
    
    if selected_site == 'ALL':
        # Scatter for all sites
        fig = px.scatter(
            filtered_df, x="Payload Mass (kg)", y="class",
            color="Booster Version Category",
            title="Payload vs. Outcome for All Sites"
        )
    else:
        # Scatter for one site
        site_df = filtered_df[filtered_df['Launch Site'] == selected_site]
        fig = px.scatter(
            site_df, x="Payload Mass (kg)", y="class",
            color="Booster Version Category",
            title=f"Payload vs. Outcome for site {selected_site}"
        )
    
    return fig

In [14]:
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard",
            style={'textAlign': 'center', 'color': '#503D36',
                   'font-size': 40}),

    # TASK 1: Dropdown
    dcc.Dropdown(
        id='site-dropdown',
        options=options,
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),

    html.Br(),

    # TASK 2: Pie chart
    dcc.Graph(id='success-pie-chart'),

    html.Br(),

    # TASK 3: Slider
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(
        id='payload-slider',
        min=0, 
        max=10000, 
        step=1000,
        marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
        value=[min_payload, max_payload]
    ),

    html.Br(),

    # TASK 4: Scatter chart
    dcc.Graph(id='success-payload-scatter-chart')
])

In [15]:
app.run(mode="inline")

In [16]:
spacex_df.groupby("Launch Site")["class"].sum().sort_values(ascending=False)

Launch Site
KSC LC-39A      10
CCAFS LC-40      7
VAFB SLC-4E      4
CCAFS SLC-40     3
Name: class, dtype: int64

In [17]:
(spacex_df.groupby("Launch Site")["class"].mean().sort_values(ascending=False))

Launch Site
KSC LC-39A      0.769231
CCAFS SLC-40    0.428571
VAFB SLC-4E     0.400000
CCAFS LC-40     0.269231
Name: class, dtype: float64

In [18]:
bins = [0, 2000, 4000, 6000, 8000, 10000]
labels = ['0–2k', '2k–4k', '4k–6k', '6k–8k', '8k–10k']
spacex_df['Payload Bin'] = pd.cut(spacex_df['Payload Mass (kg)'], bins, labels=labels)

In [20]:
# Success rate by payload bin
spacex_df.groupby("Payload Bin", observed=True)["class"].mean()

Payload Bin
0–2k      0.272727
2k–4k     0.619048
4k–6k     0.384615
6k–8k     0.000000
8k–10k    0.600000
Name: class, dtype: float64